In [1]:
import glob
import os
import math
import time
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

In [2]:
DATA_DIR = "../data_generation"               
FILE_PATTERN = "quad_data_run_*.csv"

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 1024
LR = 1e-3 
PRETRAIN_EPOCHS = 2000       # train data-only first
PHYS_RAMP_EPOCHS = 2000      # ramp physics weight gradually
TOTAL_EPOCHS = PRETRAIN_EPOCHS + PHYS_RAMP_EPOCHS + 4000
PRINT_EVERY = 200

In [3]:
# Physics weights (base values; physics weight will be ramped)
LAMBDA_BASE = 1e-4
L_z = 1.0
L_trans = 1.0
L_rot = 1.0
L_kin = 1.0

In [4]:
# Quadrotor constants (same as MATLAB)
Jxx = 6.86e-5; Jyy = 9.2e-5; Jzz = 1.366e-4
m = 0.068; g = 9.81
t1 = (Jyy - Jzz) / Jxx
t2 = (Jzz - Jxx) / Jyy
t3 = (Jxx - Jyy) / Jzz

In [5]:
def safe_array(values, deg_to_rad=False, negate=False):
    arr = np.array(values, dtype=np.float64)
    
    arr[np.isnan(arr)] = 0.0
    arr[np.isinf(arr)] = 0.0

    if deg_to_rad:
        arr = arr * np.pi / 180.0
    if negate:
        arr = -arr

    arr = np.clip(arr, -1e6, 1e6)

    return arr.reshape(-1, 1).astype(np.float32)

In [6]:
csv_files = sorted(glob.glob(os.path.join(DATA_DIR, FILE_PATTERN)))
if len(csv_files) == 0:
    raise FileNotFoundError(f"No files found with pattern {FILE_PATTERN} in {DATA_DIR}")

dfs = []
for f in csv_files:
    df = pd.read_csv(f)
    dfs.append(df)
data = pd.concat(dfs, ignore_index=True)
print(f"Loaded {len(csv_files)} files, total samples = {len(data)}")

Loaded 16 files, total samples = 16000


In [7]:
nan_counts = data.isna().sum()
nan_cols = nan_counts[nan_counts > 0]

if len(nan_cols) > 0:
    print("\n⚠️ Columns containing NaN values:")
    print(nan_cols)
    print("\nRows with NaN values:")
    print(data[data.isna().any(axis=1)])
else:
    print("\n✅ No NaN values found in the dataset.")


✅ No NaN values found in the dataset.


In [8]:
inf_mask = np.isinf(data.select_dtypes(include=[np.number])).any()
inf_cols = inf_mask[inf_mask].index.tolist()

if len(inf_cols) > 0:
    print("\n⚠️ Columns containing Inf or -Inf values:")
    print(inf_cols)
    print("\nRows with Inf values:")
    print(data[np.isinf(data.select_dtypes(include=[np.number])).any(axis=1)])
else:
    print("\n✅ No Inf or -Inf values found in the dataset.")


✅ No Inf or -Inf values found in the dataset.


In [9]:
# -------------------------
# 2) Select columns + preprocessing
# -------------------------
# Column names as generated by MATLAB code earlier
# Time_s,Height_m,X_m,Y_m,Roll_deg,Pitch_deg,Yaw_deg,
# p,q,r,u,v,w,tx,ty,tz,Thrust

required_cols = ['Time_s','Height_m','X_m','Y_m','Roll_deg','Pitch_deg','Yaw_deg',
                 'p','q','r','u','v','w','tx','ty','tz','Thrust']

for c in required_cols:
    if c not in data.columns:
        raise ValueError(f"Missing column {c} in data")
    
# Inputs: time only
t_np = safe_array(data['Time_s'])

# Targets: we will predict these outputs (14)
# We'll follow the sign convention used earlier: model z = -Height_m
z_np = safe_array(data['Height_m'], negate=True)
phi_np = safe_array(data['Roll_deg'], deg_to_rad=True)
theta_np = safe_array(data['Pitch_deg'], deg_to_rad=True)
psi_np = safe_array(data['Yaw_deg'], deg_to_rad=True)

p_np = safe_array(data['p'])
q_np = safe_array(data['q'])
r_np = safe_array(data['r'])

u_np = safe_array(data['u'])
v_np = safe_array(data['v'])
w_np = safe_array(data['w'])

tx_np = safe_array(data['tx'])
ty_np = safe_array(data['ty'])
tz_np = safe_array(data['tz'])
T_np  = safe_array(data['Thrust'])

# Stack targets in a consistent order
Y_np = np.hstack([z_np, phi_np, theta_np, psi_np,
                  p_np, q_np, r_np,
                  u_np, v_np, w_np,
                  tx_np, ty_np, tz_np, T_np])  # shape (N,14)


split_idx = int(0.7 * len(t_np))
t_train, t_test = t_np[:split_idx], t_np[split_idx:]
Y_train, Y_test = Y_np[:split_idx], Y_np[split_idx:]

print(f"Train samples: {len(t_train)}")
print(f"Test samples:  {len(t_test)}")

Train samples: 11200
Test samples:  4800


In [10]:
t_scaler = StandardScaler()
Y_scaler = StandardScaler()

# Convert to numpy if tensors
if isinstance(t_train, torch.Tensor):
    t_train = t_train.detach().cpu().numpy()
if isinstance(Y_train, torch.Tensor):
    Y_train = Y_train.detach().cpu().numpy()
if isinstance(t_test, torch.Tensor):
    t_test = t_test.detach().cpu().numpy()
if isinstance(Y_test, torch.Tensor):
    Y_test = Y_test.detach().cpu().numpy()

# Fit only on training data
t_scaler.fit(t_train)
Y_scaler.fit(Y_train)

# Transform both train and test
t_train_scaled = t_scaler.transform(t_train)
t_test_scaled = t_scaler.transform(t_test)
Y_train_scaled = Y_scaler.transform(Y_train)
Y_test_scaled = Y_scaler.transform(Y_test)

In [11]:
def to_tensor_safe(x, device="cuda"):
    if isinstance(x, np.ndarray):
        return torch.tensor(x, dtype=torch.float32, device=device)
    elif isinstance(x, torch.Tensor):
        return x.clone().detach().to(torch.float32).to(device)
    else:
        raise TypeError(f"Unsupported type for tensor conversion: {type(x)}")

# Convert to tensors
t_train_tensor = to_tensor_safe(t_train_scaled, DEVICE)
Y_train_tensor = to_tensor_safe(Y_train_scaled, DEVICE)
t_test_tensor  = to_tensor_safe(t_test_scaled, DEVICE)
Y_test_tensor  = to_tensor_safe(Y_test_scaled, DEVICE)

# Create Datasets
train_dataset = TensorDataset(t_train_tensor, Y_train_tensor)
test_dataset  = TensorDataset(t_test_tensor,  Y_test_tensor)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader  = DataLoader(test_dataset,  batch_size=BATCH_SIZE, shuffle=False, drop_last=False)

print(f"Train batches: {len(train_loader)} | Test batches: {len(test_loader)}")

Train batches: 10 | Test batches: 5


In [12]:
# -------------------------
# 4) Define PINN model
# -------------------------
class PINNNet(nn.Module):
    def __init__(self, hidden=256, layers=4, in_dim=1, out_dim=14):
        super().__init__()
        dims = [in_dim] + [hidden]*layers + [out_dim]
        net = []
        for i in range(len(dims)-1):
            net.append(nn.Linear(dims[i], dims[i+1]))
            if i < len(dims)-2:
                net.append(nn.Tanh())
        self.net = nn.Sequential(*net)
        
        # Initialize weights properly
        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            nn.init.xavier_uniform_(m.weight)
            nn.init.zeros_(m.bias)

    def forward(self, t):
        # Ensure correct input shape
        if t.ndim == 1:
            t = t.unsqueeze(-1)
        return self.net(t)


# Instantiate model, optimizer, and loss
model = PINNNet(hidden=256, layers=4, in_dim=1, out_dim=Y_train_tensor.shape[1]).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
criterion = nn.MSELoss()

In [15]:
def physics_residuals(model, t_batch, Y_scaler, t_scaler, device=DEVICE):
    """
    Compute physics-based residuals in *physical units*.
    t_batch: scaled time tensor with requires_grad=True (or must be set by caller)
    """
    # Ensure correct device and grad flag (caller should set requires_grad_)
    t_batch = t_batch.clone().detach().to(device).requires_grad_(True)

    # Forward pass: scaled outputs (keeps graph)
    pred_scaled = model(t_batch)  # (B,14)

    # Compute d(y_scaled)/d(t_scaled) for each output via autograd
    grads_scaled = []
    for i in range(pred_scaled.shape[1]):
        g = torch.autograd.grad(pred_scaled[:, i:i+1], t_batch,
                                grad_outputs=torch.ones_like(pred_scaled[:, i:i+1]),
                                retain_graph=True, create_graph=True)[0]
        grads_scaled.append(g)
    grads_scaled = torch.cat(grads_scaled, dim=1)  # (B,14)

    # Chain rule: map scaled derivatives to physical derivatives
    scale_t = float(t_scaler.scale_[0])
    scale_y = torch.tensor(Y_scaler.scale_, dtype=torch.float32, device=device)
    dy_dt_phys = grads_scaled * (scale_y / scale_t)

    # Unscale predicted outputs to physical units
    pred_phys = pred_scaled * scale_y + torch.tensor(Y_scaler.mean_, dtype=torch.float32, device=device)

    # Split outputs and derivatives
    z, phi, theta, psi, p, q, r, u, v, w, tx, ty, tz, Th = torch.split(pred_phys, 1, dim=1)
    z_t, phi_t, theta_t, psi_t, p_t, q_t, r_t, u_t, v_t, w_t, *_ = torch.split(dy_dt_phys, 1, dim=1)

    # Physical constants
    g_const = 9.81
    m = 0.068
    Jxx, Jyy, Jzz = 6.86e-5, 9.2e-5, 1.366e-4
    t1, t2, t3 = (Jyy - Jzz) / Jxx, (Jzz - Jxx) / Jyy, (Jxx - Jyy) / Jzz

    # Residual definitions (physical units)
    f_z = z_t + (-torch.sin(theta)*u + torch.cos(theta)*torch.sin(phi)*v + torch.cos(theta)*torch.cos(phi)*w)
    f_u = u_t - ( r*v - q*w - g_const*torch.sin(theta) - 0.1*u )
    f_v = v_t - ( p*w - r*u + g_const*torch.cos(theta)*torch.sin(phi) - 0.1*v )
    f_w = w_t - ( q*u - p*v + g_const*torch.cos(theta)*torch.cos(phi) - 0.1*w + (-Th)/m )

    f_p = p_t - ( t1 * q * r + tx / Jxx - 2.0 * p )
    f_q = q_t - ( t2 * p * r + ty / Jyy - 2.0 * q )
    f_r = r_t - ( t3 * p * q + tz / Jzz - 2.0 * r )

    f_phi = phi_t - ( p + torch.sin(phi)*torch.tan(theta)*q + torch.cos(phi)*torch.tan(theta)*r )
    f_theta = theta_t - ( torch.cos(phi)*q - torch.sin(phi)*r )
    f_psi = psi_t - ( torch.sin(phi)/torch.cos(theta)*q + torch.cos(phi)/torch.cos(theta)*r )

    return {
        'f_z': f_z,
        'f_trans': torch.cat([f_u, f_v, f_w], dim=1),
        'f_rot': torch.cat([f_p, f_q, f_r], dim=1),
        'f_kin': torch.cat([f_phi, f_theta, f_psi], dim=1)
    }

In [ ]:
train_losses = []
phys_losses = []
data_losses = []
val_losses = []

start_time = time.time()

# quick safety
assert len(train_loader) > 0, "train_loader is empty"
assert len(test_loader) > 0, "test_loader is empty"

for epoch in range(1, TOTAL_EPOCHS + 1):
    model.train()
    epoch_data_loss = 0.0
    epoch_phys_loss = 0.0
    n_batches = 0

    # Physics lambda schedule
    if epoch <= PRETRAIN_EPOCHS:
        lambda_phys = 0.0
    elif epoch <= PRETRAIN_EPOCHS + PHYS_RAMP_EPOCHS:
        frac = (epoch - PRETRAIN_EPOCHS) / PHYS_RAMP_EPOCHS
        lambda_phys = LAMBDA_BASE * frac
    else:
        lambda_phys = LAMBDA_BASE

    # Training loop (batches)
    for t_batch, y_batch in train_loader:
        n_batches += 1
        t_batch = t_batch.clone().detach().to(DEVICE).requires_grad_(True)   # need grad for physics
        y_batch = y_batch.to(DEVICE)

        optimizer.zero_grad()
        pred_scaled = model(t_batch)

        # Data loss
        loss_data = criterion(pred_scaled, y_batch)

        # Physics residuals (requires autograd)
        residuals = physics_residuals(model, t_batch, Y_scaler, t_scaler, DEVICE)
        loss_z = torch.mean(residuals['f_z']**2)
        loss_trans = torch.mean(residuals['f_trans']**2)
        loss_rot = torch.mean(residuals['f_rot']**2)
        loss_kin = torch.mean(residuals['f_kin']**2)
        loss_phys = L_z*loss_z + L_trans*loss_trans + L_rot*loss_rot + L_kin*loss_kin

        # Combine and step
        loss_total = loss_data + lambda_phys * loss_phys
        loss_total.backward()
        optimizer.step()

        epoch_data_loss += loss_data.item()
        epoch_phys_loss += loss_phys.item()

    # Avoid division by zero
    epoch_data_loss = epoch_data_loss / n_batches if n_batches > 0 else float('nan')
    epoch_phys_loss = epoch_phys_loss / n_batches if n_batches > 0 else float('nan')
    train_losses.append(epoch_data_loss + lambda_phys * epoch_phys_loss)
    data_losses.append(epoch_data_loss)
    phys_losses.append(epoch_phys_loss)

    # -----------------------------
    # Validation / Test evaluation
    # -----------------------------
    model.eval()
    val_data_loss = 0.0
    val_phys_loss = 0.0
    val_batches = 0

    for t_val, y_val in test_loader:
        val_batches += 1
        # Move to device (no grad yet)
        t_val = t_val.clone().detach().to(DEVICE)
        y_val = y_val.to(DEVICE)

        # ---- data loss: compute without grad for speed ----
        with torch.no_grad():
            pred_val = model(t_val)
            loss_val_data = criterion(pred_val, y_val)
            val_data_loss += loss_val_data.item()

        # ---- physics loss: need gradients for time-derivatives ----
        # create a gradient-enabled copy of t_val and compute residuals
        t_val_grad = t_val.clone().detach().requires_grad_(True)
        residuals_val = physics_residuals(model, t_val_grad, Y_scaler, t_scaler, DEVICE)
        loss_val_phys = (L_z*torch.mean(residuals_val['f_z']**2) +
                         L_trans*torch.mean(residuals_val['f_trans']**2) +
                         L_rot*torch.mean(residuals_val['f_rot']**2) +
                         L_kin*torch.mean(residuals_val['f_kin']**2))
        # .item() is safe here because loss_val_phys is scalar tensor
        val_phys_loss += loss_val_phys.item()

    # Normalize validation losses
    val_data_loss = val_data_loss / val_batches if val_batches > 0 else float('nan')
    val_phys_loss = val_phys_loss / val_batches if val_batches > 0 else float('nan')
    val_losses.append(val_data_loss + lambda_phys * val_phys_loss)

    # Logging
    if epoch % PRINT_EVERY == 0 or epoch == 1:
        elapsed = time.time() - start_time
        print(f"Epoch {epoch}/{TOTAL_EPOCHS} | "
              f"train_total={train_losses[-1]:.6e} | train_data={epoch_data_loss:.6e} | train_phys={epoch_phys_loss:.6e} | "
              f"val_total={val_losses[-1]:.6e} | val_data={val_data_loss:.6e} | val_phys={val_phys_loss:.6e} | "
              f"lambda={lambda_phys:.2e} | elapsed={elapsed/60:.2f} min")

Epoch 1/8000 | train_total=9.322726e-01 | train_data=9.322726e-01 | train_phys=2.987708e+02 | val_total=1.321252e+00 | val_data=1.321252e+00 | val_phys=2.247320e+02 | lambda=0.00e+00 | elapsed=0.08 min
Epoch 200/8000 | train_total=8.777571e-01 | train_data=8.777571e-01 | train_phys=1.852512e+02 | val_total=1.318595e+00 | val_data=1.318595e+00 | val_phys=1.828982e+02 | lambda=0.00e+00 | elapsed=17.67 min
Epoch 400/8000 | train_total=8.730793e-01 | train_data=8.730793e-01 | train_phys=1.776133e+02 | val_total=1.328512e+00 | val_data=1.328512e+00 | val_phys=2.446159e+02 | lambda=0.00e+00 | elapsed=30.97 min
Epoch 600/8000 | train_total=8.704816e-01 | train_data=8.704816e-01 | train_phys=1.843094e+02 | val_total=1.336181e+00 | val_data=1.336181e+00 | val_phys=1.650399e+02 | lambda=0.00e+00 | elapsed=42.92 min
Epoch 800/8000 | train_total=8.733274e-01 | train_data=8.733274e-01 | train_phys=1.928491e+02 | val_total=1.335295e+00 | val_data=1.335295e+00 | val_phys=1.932756e+02 | lambda=0.00e+0

In [ ]:
# -------------------------
# 7) Save model and scalers
# -------------------------
torch.save({
    'model_state_dict': model.state_dict(),
    't_scaler_mean': t_scaler.mean_,
    't_scaler_scale': t_scaler.scale_,
    'Y_scaler_mean': Y_scaler.mean_,
    'Y_scaler_scale': Y_scaler.scale_
}, 'pinn_quad_model.pth')
print("Saved model to pinn_quad_model.pth")

# -------------------------
# 8) Diagnostics plots
# -------------------------
plt.figure(figsize=(8,4))
plt.plot(train_losses, label='total (data + lambda*phys)')
plt.plot(data_losses, label='data loss')
plt.plot(phys_losses, label='phys loss')
plt.yscale('log')
plt.legend()
plt.xlabel('Epochs (binned)')
plt.title('Training losses (log scale)')
plt.show()

# Quick parity plot for a subset
model.eval()
with torch.no_grad():
    # pick first 2000 points for parity check
    Ncheck = min(2000, t_tensor.shape[0])
    t_check = t_tensor[:Ncheck].clone().to(DEVICE).requires_grad_(False)
    pred_scaled_check = model(t_check)
    pred_phys_check = pred_scaled_check.cpu().numpy() * Y_scaler.scale_ + Y_scaler.mean_
    true_phys = Y_np[:Ncheck,:]

fig, axs = plt.subplots(4,1, figsize=(8,10))
axs[0].plot(true_phys[:,0], label='z_true'); axs[0].plot(pred_phys_check[:,0], label='z_pred'); axs[0].legend()
axs[1].plot(true_phys[:,1]*180/np.pi, label='phi_true'); axs[1].plot(pred_phys_check[:,1]*180/np.pi, label='phi_pred'); axs[1].legend()
axs[2].plot(true_phys[:,2]*180/np.pi, label='theta_true'); axs[2].plot(pred_phys_check[:,2]*180/np.pi, label='theta_pred'); axs[2].legend()
axs[3].plot(true_phys[:,3]*180/np.pi, label='psi_true'); axs[3].plot(pred_phys_check[:,3]*180/np.pi, label='psi_pred'); axs[3].legend()
plt.tight_layout(); plt.show()